# tutorial https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

## Note to Jose
Working version but not fully understood yet
The next commented think can allow to remove stuff

In [1]:
# Download TorchVision repo to use some files from
# references/detection
!git clone git@github.com:jcmayoral/PyTorch-YOLOv3.git yolov3

fatal: destination path 'yolov3' already exists and is not an empty directory.


In [2]:
from yolov3.models import Darknet

#model_4classes = Darknet(config_path="yolov3/config/yolov3-custom.cfg")

In [3]:
#nparams = sum(p.numel() for p in model_4classes.parameters())
#print (f"{nparams:,}")

In [4]:
bs = 1
nepochs = 50
ncpu = 1
gradient_accumulations = 2
evaluation_interval = 10
checkpoint_interval = 10
img_size = 128

In [5]:
from __future__ import division

from yolov3.models import *
from yolov3.utils.logger import *
from yolov3.utils.utils import *
from yolov3.utils.datasets import *
from yolov3.utils.parse_config import *
from yolov3.test import evaluate

from terminaltables import AsciiTable

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim


"""
parser.add_argument("--epochs", type=int, default=100, help="number of epochs")
parser.add_argument("--batch_size", type=int, default=8, help="size of each image batch")
parser.add_argument("--gradient_accumulations", type=int, default=2, help="number of gradient accums before step")
parser.add_argument("--model_def", type=str, default="config/yolov3.cfg", help="path to model definition file")
parser.add_argument("--data_config", type=str, default="config/coco.data", help="path to data config file")
parser.add_argument("--pretrained_weights", type=str, help="if specified starts from checkpoint model")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
parser.add_argument("--evaluation_interval", type=int, default=1, help="interval evaluations on validation set")
parser.add_argument("--compute_map", default=False, help="if True computes mAP every tenth batch")
parser.add_argument("--multiscale_training", default=True, help="allow for multi-scale training")
opt = parser.parse_args()
print(opt)
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("output", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

# Get data configuration
data_config = parse_data_config("yolov3/config/iros2021-rgbd.data")
#data_config = { line.split()[0] : line.split()[1] for line in open("yolov3/config/jose.data") }

#parse_data_config(opt.data_config)bb
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

print(class_names)

# Initiate model
#model = Darknet("yolov3/config/iros2021-rgbd.cfg").to(device)#, img_size=img_size).to(device)
model = Darknet("yolov3/config/iros2021-tiny-rgbd.cfg").to(device)#, img_size=img_size).to(device)
model.apply(weights_init_normal)

# If specified we start from checkpoint





/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

['ring0', 'ring1', 'ring2', 'ring3']


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (maxpool_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (4): Sequential(
      (conv_4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_4): BatchNorm2d(64, eps=1e-05, momentum=0.9, aff

In [6]:
#model.load_state_dict(torch.load("/home/jose/ros_ws/src/gr_perception/gr_ml/nb/testingyolo/checkpoints/yolov3_v2_ckpt_140.pth"))

In [7]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/yolo_rgbd')
plot_interval = 100

In [ ]:
# Get dataloader
#For some reason augment or multiscale fails
dataset = MyListDataset(train_path, augment=True, multiscale=True)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=bs,
    shuffle=True,
    num_workers=ncpu,
    pin_memory=False,
    collate_fn=dataset.collate_fn,
)

optimizer = torch.optim.Adam(model.parameters())

metrics = [
    "grid_size",
    "loss",
    "x",
    "y",
    "w",
    "h",
    "conf",
    "cls",
    "cls_acc",
    "recall50",
    "recall75",
    "precision",
    "conf_obj",
    "conf_noobj",
]

try:
    os.makedirs("depth_testingyolo")
except:
    pass

#logger = Logger("testingyolo/logs")


for epoch in range(nepochs):
    model.train()
    start_time = time.time()
    epoch_loss = 0.0

    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        #print(_)0
        batches_done = len(dataloader) * epoch + batch_i
        print(imgs.shape)

        imgs = Variable(imgs.to(device))
        targets = Variable(targets.to(device), requires_grad=False)

        loss, outputs = model(imgs, targets)
        loss.backward()
        
        epoch_loss += outputs.shape[0] * loss.item()

        writer.add_scalar('Loss/train_batch', loss,(epoch*len(dataloader)+batch_i))

        if batches_done % gradient_accumulations:
            # Accumulates gradient before each step
            optimizer.step()
            optimizer.zero_grad()

        # ----------------
        #   Log progress
        # ----------------
        yololoss = [yolo.metrics["loss"] for yolo in model.yolo_layers]
        for i, yloss in enumerate(yololoss):
            writer.add_scalar('Loss/yolo_loss'+str(i), yloss,(epoch*len(dataloader)+batch_i))
        #writer.add_scalar('Loss/yolo_loss1', yololoss[1],(epoch*len(dataloader)+batch_i))
        #writer.add_scalar('Loss/yolo_loss2', yololoss[2],(epoch*len(dataloader)+batch_i))

        log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, nepochs, batch_i, len(dataloader))

        metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

        if batch_i % plot_interval == 0:
            # Log metrics at each YOLO layer
            for i, metric in enumerate(metrics):
                formats = {m: "%.6f" for m in metrics}
                formats["grid_size"] = "%2d"
                formats["cls_acc"] = "%.2f%%"
                row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
                metric_table += [[metric, *row_metrics]]

                # Tensorboard logging
                """
                tensorboard_log = []
                for j, yolo in enumerate(model.yolo_layers):
                    for name, metric in yolo.metrics.items():
                        if name != "grid_size":
                            tensorboard_log += [(f"{name}_{j+1}", metric)]
                tensorboard_log += [("loss", loss.item())]
                """
                #logger.list_of_scalars_summary(tensorboard_log, batches_done)

            log_str += AsciiTable(metric_table).table
            log_str += f"\nTotal loss {loss.item()}"

            # Determine approximate time left for epoch
            epoch_batches_left = len(dataloader) - (batch_i + 1)
            time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
            log_str += f"\n---- ETA {time_left}"

            print(log_str)
            #break

        model.seen += imgs.size(0)
    
    
    writer.add_scalar('Loss/train_epoch', epoch_loss / len(dataloader), epoch)
    #print ("HERE", len(dataloader), epoch_loss)


    if epoch % evaluation_interval == 0:
        print(log_str)
        print("\n---- Evaluating Model ----")
        # Evaluate the model on the validation set
        eval_results = evaluate(
            model,
            path=valid_path,
            iou_thres=0.5,
            conf_thres=0.5,
            nms_thres=0.5,
            img_size=img_size,
            batch_size=1,
        )
        
        if eval_results is not None:
            precision, recall, AP, f1, ap_class = eval_results
            """
            evaluation_metrics = [
                ("val_precision", precision.mean()),
                ("val_recall", recall.mean()),
                ("val_mAP", AP.mean()),
                ("val_f1", f1.mean()),
            ]
            logger.list_of_scalars_summary(evaluation_metrics, epoch)
            """
            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
            print(f"---- mAP {AP.mean()}")
            writer.add_scalar('metrics/map_eval', AP.mean(), int(epoch / evaluation_interval))
            writer.add_scalar('metrics/precision', precision.mean(), int(epoch / evaluation_interval))
            writer.add_scalar('metrics/recall', recall.mean(), int(epoch / evaluation_interval))
            writer.add_scalar('metrics/f1', f1.mean(), int(epoch / evaluation_interval))


    if epoch % checkpoint_interval == 0:
        print (log_str)
        torch.save(model.state_dict(), f"depth_testingyolo/checkpoints/rgbd_yolov3_v2_ckpt_%d.pth" % epoch)


torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 512, 512])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])

---- [Epoch 0/50, Batch 0/900] ----
+------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 |
+------------+--------------+--------------+
| grid_size  | 16           | 32           |
| loss       | 77.654663    | 80.307655    |
| x          | 0.365221     | 0.436783     |
| y          | 0.336193     | 0.516459     |
| w          | 0.487661     | 0.006882     |
| h          | 0.886058     | 2.589200     |
| conf       | 74.867180    | 76.056961    |
| cls        | 0.712346     | 0.701371     |
| cls_acc    | 0.00%        | 0.00%        |
| recall50   | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     |
| precisio

torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([1, 4, 352, 352])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 480, 480])

---- [Epoch 0/50, Batch 100/900] ----
+------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 |
+------------

torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 480, 480])
torch.Size([1, 4, 480, 480])
torch.Size([1, 4, 480, 480])
torch.Size([1, 4, 480, 480])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 480, 480])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([3, 480, 640]) torch.Siz

torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 4, 384, 384])
torch.Size([1, 

torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 416, 416])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([1, 4, 416, 416])
torch.Size([1, 4, 448, 448])
torch.Size([1, 4, 448, 448])
torch.Size([1, 4, 448, 448])
torch.Size([3, 480, 640]) torch.Size([480, 640])
torch.Size([3, 

In [ ]:
def get_image_iteration(myiter):
    return next(myiter)

In [ ]:
labels=[]

In [ ]:
conf_thres = 0.9
nms_thres = 0.9

In [ ]:
def evaluate(b):
    model.eval()
    with torch.no_grad():
        outputs = model(b.to(device))
        outputs = non_max_suppression(outputs, conf_thres=conf_thres, nms_thres=nms_thres)
    return outputs

In [ ]:
from matplotlib.ticker import NullLocator

cmap = plt.get_cmap("tab20b")
colors = [cmap(i) for i in np.linspace(0, 1, 20)]


In [ ]:
def plot_results(index):
    myiter = iter(dataloader)
    a,b,c = get_image_iteration(myiter)
    outputs = evaluate(b)
    # Iterate through images and save plot of detections
    for img_i, (image, detections) in enumerate(zip(b, outputs)):
        print("(%d) Image" % (img_i))

        # Create plot
        #img = np.array(Image.open(path))
        img = image.mul(255).permute(1, 2, 0).byte().cpu().numpy()
        plt.figure()
        fig, ax = plt.subplots(1)
        ax.imshow(img)

        # Draw bounding boxes and labels of detections
        if detections is not None:
            # Rescale boxes to oripginal image
            detections = rescale_boxes(detections, img_size, img.shape[:2])
            unique_labels = detections[:, -1].cpu().unique()
            n_cls_preds = len(unique_labels)
            bbox_colors = random.sample(colors, n_cls_preds)
            for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

                print("\t+ Label: %s, Conf: %.5f" % (class_names[int(cls_pred)], cls_conf.item()))

                box_w = x2 - x1
                box_h = y2 - y1

                if cls_conf.item()<0.7:
                    continue



                color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
                # Create a Rectangle patch
                bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
                # Add the bbox to the plot
                ax.add_patch(bbox)
                # Add label
                plt.text(
                    x1,
                    y1,
                    s=class_names[int(cls_pred)]+" {:.1f}".format(cls_conf.item()),
                    color="white",
                    verticalalignment="top",
                    bbox={"color": color, "pad": 0},
                )

        plt.axis("off")
        plt.gca().xaxis.set_major_locator(NullLocator())
        plt.gca().yaxis.set_major_locator(NullLocator())
        plt.savefig("depth_output/example_yolo{}_{}.png".format(str(img_i), str(index)), bbox_inches="tight", pad_inches=0.0,dpi=250)
        plt.show()
        plt.close()

In [ ]:
for j in range(10):
    plot_results(j)

In [ ]:
c